In [14]:
import os
import json
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50

# Define paths.
dataset_path = './plants-classification'
train_dir = os.path.join(dataset_path, 'train')
validation_dir = os.path.join(dataset_path, 'val')
test_dir = os.path.join(dataset_path, 'test')

# Define common parameters.
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess images from the training, validation, and test directories.
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset="training",
    shuffle=True,
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Create a model with the following sequential processing layers.
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model.
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model for 100 epochs.
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    verbose=1
)

# Save the model and training history for use in the application.
model_save_path = './plant_species_classifier.keras'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

with open('./training_history.json', 'w') as f:
    json.dump(history.history, f)

test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy: {test_acc}")

Found 21000 images belonging to 30 classes.
Found 3000 images belonging to 30 classes.
Found 6000 images belonging to 30 classes.
Epoch 1/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 191s 288ms/step - accuracy: 0.1115 - loss: 3.1950 - val_accuracy: 0.2910 - val_loss: 2.4813
Epoch 2/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 190s 288ms/step - accuracy: 0.2469 - loss: 2.5392 - val_accuracy: 0.3343 - val_loss: 2.2527
Epoch 3/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 190s 288ms/step - accuracy: 0.2987 - loss: 2.3625 - val_accuracy: 0.3570 - val_loss: 2.1752
Epoch 4/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 190s 287ms/step - accuracy: 0.3290 - loss: 2.2689 - val_accuracy: 0.4203 - val_loss: 1.9787
Epoch 5/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 190s 287ms/step - accuracy: 0.3543 - loss: 2.1734 - val_accuracy: 0.4430 - val_loss: 1.9400
Epoch 6/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 190s 288ms/step - accuracy: 0.3701 - loss: 2.1287 - val_accuracy: 0.4817 - val_loss: 1.7887
Epoch 7/100
657/657 ━━━━━━━━━━━━━━━━━━━━ 190s 287ms/step - accuracy: 0.3